# Importing & FIltering Data

### Import Basic Packages

In [23]:
#Basics
import numpy as np
import pandas as pd

### Import Data from CSV

In this scenario we'll import a dataset of students enrolled in a school and explore ways to select and filter data of interest.




In [24]:
# Import data to a pandas dataframe
df_grades = pd.read_csv('student grades.csv')
df_grades.head()

,student_ID,first_name,last_name,grade_avg,faculty,tuition,OH_participated,classes_skipped
0,20123456.0,John,Park,B,Arts,44191.0,0,5.0
1,20123457.0,Alex,Great,B,Science,32245.0,"""4""",10.0
2,20123458.0,Sebastian,Taylor,B,Business,42679.0,6,7.0
3,20123459.0,Michael,Bay,A,Math,46478.0,15,2.0
4,20123460.0,Scott,Foster,A,Engineering,36784.0,5,8.0


### Import Data from Excel File

In [25]:
# Import data to a pandas dataframe
df_grades_excel = pd.read_excel('student grades.xlsx',sheet_name='student_grades', usecols =['faculty','tuition'])
df_grades_excel.head()

,faculty,tuition
0,Arts,44191
1,Science,32245
2,Business,42679
3,Math,46478
4,Engineering,36784


### Import Data Using SQL Query

Here we have simply defined an SQL query we want to use to retrieve data from our database.

In [41]:
sql_query = """SELECT 
 
    hist.FactID,
    hist.Date,
    hist.[Open],
    hist.High,
    hist.Low,
    hist.[Close],
    hist.AdjClose,
    hist.Volume,
 
    sec.Company,
    sec.Symbol,
    sec.Industry,
    sec.IndexWeighting,
 
    exc.Symbol AS Exchange
 
FROM [dbo].[FactPrices_Daily] AS hist
 
   INNER JOIN [dbo].[dimSecurity] AS sec 
      ON hist.SecurityID = sec.ID
      
   INNER JOIN [dbo].[dimExchange] AS exc 
      ON sec.ExchangeID = exc.ID
;"""

### Import Data from SQL Database Using pyodbc & sqlalchemy (Windows only syntax)

In [42]:
import pyodbc
import os
import urllib
from sqlalchemy import create_engine
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
driver = '{ODBC Driver 18 for SQL Server}'
server = 'prod-sql-cfieducation.database.windows.net' 
database = 'StockPricesDW' 
username = 'ReportingUser'
password = 'CFICapitalPartners789#'

connection_string = f'DRIVER={driver};SERVER=tcp:{server};DATABASE={database};UID={username};PWD={password}'
odbc_params = urllib.parse.quote_plus(connection_string)
conn_string = f'mssql+pyodbc:///?odbc_connect={odbc_params}'
engine = create_engine(conn_string)

In [43]:
df_sql = pd.read_sql(sql_query, engine)
df_sql.head()

,FactID,Date,Open,High,Low,Close,AdjClose,Volume,Company,Symbol,Industry,IndexWeighting,Exchange
0,1,2015-01-02,164.71001,165.08,162.73000,164.06000,136.07068,2116400,3M,MMM,Conglomerate,0.0384,NYSE
1,2,2015-01-05,163.00000,163.64,160.08000,160.36000,133.00192,3692900,3M,MMM,Conglomerate,0.0384,NYSE
2,3,2015-01-06,160.82001,161.37,157.74001,158.64999,131.58366,3532000,3M,MMM,Conglomerate,0.0384,NYSE
3,4,2015-01-07,159.89999,160.28,158.94000,159.80000,132.53748,3081300,3M,MMM,Conglomerate,0.0384,NYSE
4,5,2015-01-08,160.64999,163.69,160.52000,163.63001,135.71404,3142900,3M,MMM,Conglomerate,0.0384,NYSE


### Selecting Columns

In [29]:
# Use Option 2 above to create a new dataframe
# Keep only the FactID and AdjClose price columns
df_small = df_grades[['first_name', 'faculty']]
df_small.head()

,first_name,faculty
0,John,Arts
1,Alex,Science
2,Sebastian,Business
3,Michael,Math
4,Scott,Engineering


In [30]:
# Use Option 2 above to create a new dataframe from the 
# This time drop the FactID and AdjClose price columns
df_small_drop = df_grades.drop(['first_name', 'faculty'], axis = 1)
df_small_drop.head()

,student_ID,last_name,grade_avg,tuition,OH_participated,classes_skipped
0,20123456.0,Park,B,44191.0,0,5.0
1,20123457.0,Great,B,32245.0,"""4""",10.0
2,20123458.0,Taylor,B,42679.0,6,7.0
3,20123459.0,Bay,A,46478.0,15,2.0
4,20123460.0,Foster,A,36784.0,5,8.0


In [31]:
# Using the filter function with regex parameters to find a column that contains a specific word
df_filter = df_grades.filter(regex='name', axis = 1)
df_filter

,first_name,last_name
0,John,Park
1,Alex,Great
2,Sebastian,Taylor
3,Michael,Bay
4,Scott,Foster
5,Amy,Winehouse
6,Ralph,Wiggins
7,Homer,Simpson
8,Marge,Simpson
9,Peter,Gryffin


### Filtering Rows

In [32]:
# Using the contains functions to find rows that contains a specific word in a column
df_contains = df_grades[df_grades["first_name"].str.contains("John")]
df_contains

,student_ID,first_name,last_name,grade_avg,faculty,tuition,OH_participated,classes_skipped
0,20123456.0,John,Park,B,Arts,44191.0,0,5.0
19,20123475.0,Johnson,Curry,B,Engineering,45000.0,9,3.0


In [45]:
# Filter the rows of the original datframe to include only rows where GradeAverage is A
df_condition = df_grades[df_grades['grade_avg'] == "A"]
df_condition

,student_ID,first_name,last_name,grade_avg,faculty,tuition,OH_participated,classes_skipped
3,20123459.0,Michael,Bay,A,Math,46478.0,15,2.0
4,20123460.0,Scott,Foster,A,Engineering,36784.0,5,8.0
11,20123467.0,Megan,Botts,A,Science,34751.0,25,1.0
12,20123468.0,Cyrus,Wong,A,Science,49298.0,20,0.0
14,20123470.0,Zachary,Chua,A,Business,NaN,10,0.0
20,20123476.0,Paul,Reed,A,Business,41397.0,14,NaN
21,20123477.0,Josh,Hart,A,Engineering,46468.0,4,8.0
22,20123478.0,Justin,Kang,A,Engineering,NaN,2,9.0
23,20123479.0,Kevin,Yoo,A,Arts,41048.0,8,1.0
24,20123480.0,Rosaline,Jun,A,Art$,44915.0,25,3.0


### Drop Duplicates

In [35]:
df_grades[df_grades['last_name'].duplicated()]

,student_ID,first_name,last_name,grade_avg,faculty,tuition,OH_participated,classes_skipped
8,20123464.0,Marge,Simpson,B,Math,NaN,"""10""",3.0
25,20123481.0,Jimin,Park,B,Engineering,40.0,20,NaN


In [36]:
#drop duplicates column
df_grades = df_grades.drop_duplicates(subset=['last_name'])
# Show how we check for dropped duplicate 
# df_grades, look for specific index that was dropped 

In [37]:
df_grades

,student_ID,first_name,last_name,grade_avg,faculty,tuition,OH_participated,classes_skipped
0,20123456.0,John,Park,B,Arts,44191.0,0,5.0
1,20123457.0,Alex,Great,B,Science,32245.0,"""4""",10.0
2,20123458.0,Sebastian,Taylor,B,Business,42679.0,6,7.0
3,20123459.0,Michael,Bay,A,Math,46478.0,15,2.0
4,20123460.0,Scott,Foster,A,Engineering,36784.0,5,8.0
5,20123461.0,Amy,Winehouse,B,Arts,36537.0,10,3.0
6,20123462.0,Ralph,Wiggins,B,Business,40762.0,2,8.0
7,20123463.0,Homer,Simpson,C,Engineering,47669.0,4,7.0
9,20123465.0,Peter,Gryffin,D,Arts,31956.0,7,7.0
10,20123466.0,Louise,King,D,Business,33227.0,6,7.0


In [38]:
# Import the phone_marketplace_dataset_cleaning_set.csv file into a dataframe
df_phone = pd.read_csv('phone_marketplace_dataset_cleaning_set.csv')
df_phone

,price,year_made,name,battery_life_percentage,storage,magnet_charging,marketplace,years_owned,visible_scratches,pro,original_sale_price,#_of_previous_owners,megapixel
0,551.0,2019,iPhone_11,74,64,no,kijiji,2,9,no,747,1,12
1,822.0,2020,iPhone_12,94,128,yes,craigslist,2,6,no,888,1,16
2,1008.0,2022,iPhone_14,97,256,yes,craigslist!,0,2,no,1185,1,22
3,NaN,2021,iPhone_13,90,128,yes,craigslist,2,2,no,887,1,20
4,839.0,2020,iPhone_12,91,256,yes,kijiji,1,5,no,969,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,1326.0,2022,iPhone_14,91,64,yes,craigslist,0,0,no,1394,1,22
345,458.0,2019,iPhone_11,75,256,no,facebook,3,3,no,702,2,12
346,487.0,2019,iPhone_11,87,256,no,facebook,1,7,no,781,2,12
347,1340.0,2022,iPhone_14,100,256,yes,craigslist,0,0,no,1411,1,22


### Conditional Filtering

In [39]:
# Filter the data that has craigstlist as marketplace
df_phone[df_phone['marketplace'] == 'craigslist']

,price,year_made,name,battery_life_percentage,storage,magnet_charging,marketplace,years_owned,visible_scratches,pro,original_sale_price,#_of_previous_owners,megapixel
1,822.0,2020,iPhone_12,94,128,yes,craigslist,2,6,no,888,1,16
3,NaN,2021,iPhone_13,90,128,yes,craigslist,2,2,no,887,1,20
8,1090.0,2024,iPhone_14,92,128,yes,craigslist,0,1,no,1167,1,22
9,1049.0,2022,iPhone_14,94,128,yes,craigslist,0,1,no,1150,1,22
10,507.0,2019,iPhone_11,74,128,no,craigslist,4,10,no,519,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,670.0,2020,iPhone_12,83,128,yes,craigslist,1,0,no,685,1,16
342,1231.0,2023,iPhone_14,97,256,yes,craigslist,0,2,no,1244,1,22
343,NaN,2022,iPhone_14,99,64,yes,craigslist,0,0,yes,1491,1,22
344,1326.0,2022,iPhone_14,91,64,yes,craigslist,0,0,no,1394,1,22


In [40]:
# Add an additional filter for iphone 11 only.
df_phone[(df_phone['marketplace'] == 'craigslist') & (df_phone["name"].str.contains("11"))] 

,price,year_made,name,battery_life_percentage,storage,magnet_charging,marketplace,years_owned,visible_scratches,pro,original_sale_price,#_of_previous_owners,megapixel
10,507.0,2019,iPhone_11,74,128,no,craigslist,4,10,no,519,1,12
17,NaN,2019,iPhone_11,87,128,no,craigslist,4,1,no,738,4,12
47,551.0,2019,iPhone_11,76,128,no,craigslist,3,3,no,636,1,12
56,576.0,2019,iPhone_11,71,64,no,craigslist,1,0,no,765,2,12
63,501.0,2019,iPhone_11,83,64,no,craigslist,3,1,no,680,1,12
99,557.0,2019,iPhone_11,72,128,no,craigslist,1,6,no,739,2,12
107,446.0,2019,iPhone_11,90,256,no,craigslist,3,3,no,774,3,12
111,436.0,2019,iPhone_11,72,256,no,craigslist,4,8,no,602,1,12
116,556.0,2019,iPhone_11,73,128,no,craigslist,1,1,no,572,1,12
119,542.0,2019,iPhone_11,79,128,no,craigslist,4,7,no,728,2,12
